In [1]:
from requests import request
from requests.compat import urljoin, urlparse
from requests.exceptions import HTTPError
from urllib.robotparser import RobotFileParser
from bs4 import BeautifulSoup
from time import sleep
import re

In [2]:
def canfetch(url, agent='*', path='/'):
    robot = RobotFileParser(urljoin(url, '/robots.txt'))
    robot.read()
    return robot.can_fetch(agent, urlparse(url)[2])
    
def download(url, params={}, headers={}, method='GET', limit=3):
    if canfetch(url) == False:
        print('[Error] ' + url)
#     else: # 실제 수집할 때, 제약사항이 많으므로 여기선 잠시 해제
    try:
        resp = request(method, url,
               params=params if method=='GET' else {},
               data=params if method=='POST' else {},
               headers=headers)
        resp.raise_for_status()
    except HTTPError as e:
        if limit > 0 and e.response.status_code >= 500:
            print(limit)
            time.sleep(1) # => random
            resp = download(url, params, headers, method, limit-1)
        else:
            print('[{}] '.format(e.response.status_code) + url)
            print(e.response.status_code)
            print(e.response.reason)
            print(e.response.headers)
    return resp

### Naver Crawler

In [3]:
url = 'https://search.naver.com/search.naver'
params = {
    'sm':'top_hty',
    'fbm':0,
    'ie':'utf8',
    'query':''
}
params['query'] = '파 이 썬'
headers = {
    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'
}
resp = download(url, params, headers, 'GET')
dom = BeautifulSoup(resp.text, 'html.parser')
urls = [{'url':_['href'], 'depth':1}
       for _ in dom.select('div.blog.section._blogBase._prs_blg dt > a[href]')]

[Error] https://search.naver.com/search.naver


In [4]:
seen = list()
# urls.append(url)

while urls: # Queue
    seed = urls.pop(-1) # starting url
    seen.append(seed) # => 재방문 회피
    if seed['depth'] < 3: # 범위 제한 => Focused Crawling
        dom = BeautifulSoup(download(seed['url']).text, 'html.parser') # HTTP
        for _ in [_['href'] if _.has_attr('href') else _['src'] for _ in dom.select('a[href], #mainFrame') 
                  if re.match(r'(?:https?:/)?/\w+(?:[./]\w+)+', _['href'] if _.has_attr('href') else _['src'])]:
                newUrls = urljoin(seed['url'], _)
                if urlparse(newUrls)[1] in ['blog.naver.com'] \
                and newUrls not in [_['url'] for _ in seen] \
                and newUrls not in [_['url'] for _ in urls]: # domain 제한
                    urls.append({'url' : newUrls, 'depth' : seed['depth']+1})
        print(len(urls), len(seen))

4 1
14 2
3 14
11 15
2 25
6 26
1 32
9 33


#### Page Ranking

In [16]:
import sqlite3

con = sqlite3.connect('pagerank.db')
cur = con.cursor()

In [17]:
cur.executescript('''
    DROP TABLE IF EXISTS host;
    CREATE TABLE host(
        pk      INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        netloc  TEXT NOT NULL
    );
    
    DROP TABLE IF EXISTS url;
    CREATE TABLE url(
        pk      INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        fk      INTEGER NOT NULL,
        ref     INTEGER NOT NULL,
        path    TEXT NOT NULL,
        crawled TEXT NOT NULL
    )
''')

In [18]:
url = 'https://www.google.com/search'
params = {
    'q':'',
    'oq':'',
    'aqs':'chrome..69i57j69i59j69i65l3j69i61j69i60j69i61.1205j0j7',
    'sourceid':'chrome',
    'ie':'UTF-8'
}
params['q'] = params['oq'] = '파이썬'
headers = {
    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'
}
resp = download(url, params, headers, 'GET')
dom = BeautifulSoup(resp.text, 'html.parser')

for _ in dom.select('div.r > a[href]'):
    urlseg = urlparse(_['href'])
    # url => http://netloc/path?query
    # netloc = htttp://netloc
    netloc = '://'.join(urlseg[:2])
    # path = path?query
    path = urlseg[2]+('?'+urlseg[4] if urlseg[4] else '')
    # host테이블에 netloc이 있는지 확인
    cur.execute('SELECT pk FROM host WHERE netloc=? ORDER BY pk ASC LIMIT 0,1',
                ['://'.join(urlseg[:2]),])
    # 없다면,
    if not cur.fetchone():
        # host테이블에 netloc 추가
        cur.execute('INSERT INTO host(netloc) VALUES(?)',
                    ['://'.join(urlseg[:2]),])
        con.commit()
    # url테이블에 path 추가 (단, host테이블 netloc의 pk를 fk로 가져옴)
    cur.execute('''
        INSERT INTO url(fk, ref, path, crawled)
        VALUES((SELECT pk FROM host WHERE netloc=? ORDER BY pk LIMIT 0, 1), ?, ?, 'N')
    ''', [netloc, 0, path])
    con.commit()

[Error] https://www.google.com/search


In [19]:
while range(10):
    cur.execute('''
        SELECT host.pk, url.pk, host.netloc, url.path FROM url
        INNER JOIN host ON host.pk=url.fk
        WHERE url.crawled='N'
        ORDER BY url.pk ASC
        LIMIT 0,1
    ''')
    seed = cur.fetchone()

    if not seed:
        break
        
    cur.execute('''
        UPDATE url SET crawled='Y' WHERE pk=?
    ''', [seed[1],])
    url = ''.join(seed[2:])
    dom = BeautifulSoup(download(url).text, 'html.parser')
    for _ in [_['href'] for _ in dom.select('a[href]') if re.match(r'(?:https?:/)?/\w+(?:[./]\w+)+',_['href'])]:
        urlseg = urlparse(urljoin(url, _))
        netloc = '://'.join(urlseg[:2])
        path = urlseg[2]+('?'+urlseg[4] if urlseg[4] else '')
        cur.execute('''
            SELECT pk FROM host WHERE netloc=?
            ORDER BY pk ASC LIMIT 0,1''', ['://'.join(urlseg[:2]),])
        if not cur.fetchone():
            cur.execute('INSERT INTO host(netloc) VALUES(?)',
                        ['://'.join(urlseg[:2]),])
            con.commit()
            
        cur.execute('''
            SELECT count(pk) FROM url WHERE fk=? and path=?
        ''', [seed[0], path])
        
        if cur.fetchone()[0] < 1:
            cur.execute('''
            INSERT INTO url(fk, ref, path, crawled)
            VALUES((SELECT pk FROM host WHERE netloc=? ORDER BY pk LIMIT 0, 1), ?, ?, 'N')
            ''', [netloc, seed[0], path])
            con.commit()
    break

In [24]:
cur.execute('SELECT COUNT(pk) FROM host')
cnt = cur.fetchone()[0]
print(cnt)

131


In [61]:
import numpy as np

A = np.zeros((cnt+1, cnt+1))
V = np.zeros((cnt+1, ))
S = np.zeros((cnt+1, ))

A.shape, v.shape, S.shape

((132, 132), (132,), (132,))

In [64]:
for j in range(1, cnt+1):
    cur.execute('''
        SELECT fk, count(fk) FROM url
        WHERE ref=? GROUP BY fk
        ''', [j,])
    
    for i in cur:
        A[i[0],j] = i[1]
        S[j] += i[1]

In [66]:
# A /= S
# A[np.isnan(A)] = 0
# A.dot(V)

### Scraping

In [128]:
url = 'http://www.ppomppu.co.kr/zboard/zboard.php?'
params = {
    'id':'ppomppu',
    'page' : 1
}
headers = {
    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'
}
resp = download(url, params, headers, 'GET')
dom = BeautifulSoup(resp.text, 'lxml')

In [125]:
# Scraping
title = re.compile(r'[[]?(\w+)[]]?([^(]+)[(](.+?)[)]')
'''
[[]? => (문자)[있고, 없고
 (\w+) => 글자 1개 이상
 []]? => (문자)] 있고, 없고
([^(]+) => (그룹) (문자)(가 아닌 다른 문자가 1개 이상 (2)
  [(] => (문자)( 있고
    (.+?) => 아무글자나 1개 이상(Lazy) (3)
 ])] => (문자)) 있고
'''

for _ in dom.select('#revolution_main_table tr[class^=list]')[1:]:
    
#     print(td[3].text.strip(), td[-2].text.strip())
    item = dict()
    td = _.find_all('td')
    frag1 = title.findall(td[3].text)[0]
    frag2 = td[-2].text.split('-')
    
    item['mall'] = frag1[0].strip()
    item['title'] = frag1[1].strip()
    item['price'] = frag1[2].strip()
    item['tu'] = frag2[0].strip() if len(frag2) > 1 else 0
    item['td'] = frag2[1].strip() if len(frag2) > 1 else 0
    
    print(item)

#     print(title.findall(td[3].text))
#     print(td[-2].text.split('/'))
#     print([_.text.strip() for _ in _.find_all('td')])
#     print([_.find_parent().find_parent().name for _ in _.find_all('td')])
#     break

{'mall': '옥션', 'title': 'LG 통돌이 TR12BL', 'price': '36100원 무료', 'tu': 0, 'td': 0}
{'mall': '무신사', 'title': '스탠다드 깅엄 체크 린넨 셔츠', 'price': '9,900원/무료', 'tu': 0, 'td': 0}
{'mall': '지마켓', 'title': '말로 때린 상처가 더 아프다', 'price': '11480원/무배', 'tu': 0, 'td': 0}
{'mall': '쿠팡', 'title': '스윙칩 60g 12개', 'price': '9,900원 무료배송', 'tu': '1', 'td': '0'}
{'mall': 'G마켓', 'title': '컬쳐랜드 5만원권 8%', 'price': '46,000/무료', 'tu': '3', 'td': '1'}
{'mall': '티몬', 'title': '[끌올] 머지포인트', 'price': '89,900원/무료', 'tu': 0, 'td': 0}
{'mall': '비밀의공구', 'title': '듀얼브레서 프리다이빙 v3 스노쿨링 마스크', 'price': '59,800 / 3,000', 'tu': 0, 'td': 0}
{'mall': '동아제약', 'title': '데일리 마스크 50매', 'price': '페이코할인+몰신규가입시 14,235 / 무배', 'tu': 0, 'td': 0}
{'mall': '11번가', 'title': '원더스리빙 WD2200 헤어드라이기 2200W', 'price': '9900/무료', 'tu': '1', 'td': '0'}
{'mall': '위메프', 'title': '1.3kg 르누아르 노트북 모던14 4500u', 'price': '633,480원 / 무료', 'tu': 0, 'td': 0}
{'mall': 'G마켓', 'title': 'KFAD 의약외품 국산 비말차단마스크 50매 휘게데이즈', 'price': '28,900/무배', 'tu': 0, 'td': 0}
{'mall': 'G마

In [136]:
# Crawling[링크] + Scraping[내용]
# Focused Crawling - 게시판의 파라미터로 제한
for page in range(1,10):
    params['page'] = page
    resp = download(url, params, headers, 'GET')
    dom = BeautifulSoup(resp.text, 'lxml')
    
    # Scraping
    for _ in dom.select('#revolution_main_table tr[class^=list]')[1:]:
        item = dict()
        td = _.find_all('td', recursive=False)
        temp = title.findall(td[3].text)
        frag1 = temp[0] if len(temp) > 0 else td[3].text.strip()
        frag2 = td[-2].text.split('-')

        item['mall'] = frag1[0].strip()
        item['title'] = frag1[1].strip()
        item['price'] = frag1[2].strip()
        item['tu'] = frag2[0].strip() if len(frag2) > 1 else 0
        item['td'] = frag2[1].strip() if len(frag2) > 1 else 0
        print(item)
#         break

{'mall': '위메프', 'title': '애플워치3 38mm GPS 알루미늄', 'price': '247,820/무료', 'tu': 0, 'td': 0}
{'mall': '개근질마트', 'title': '비앙코 볼토 초고속 블렌더', 'price': '진공', 'tu': 0, 'td': 0}
{'mall': 'G마켓', 'title': '니베아 맨 딥 쉐이빙젤 200ml 1+1. 스마일클럽, 화요카드만', 'price': '9,120/2500', 'tu': '2', 'td': '0'}
{'mall': 'G마켓', 'title': '전자렌지 용기 400ml 32개', 'price': '스마일클럽 10,790원/무료', 'tu': '1', 'td': '0'}
{'mall': '위메프', 'title': '슈퍼치즈 왕 돈까스200G', 'price': '2,850/무료', 'tu': 0, 'td': 0}
{'mall': '원더폰', 'title': '갤럭시 A10e 단순박스개봉 미사용폰', 'price': '130,000/무료', 'tu': 0, 'td': 0}
{'mall': '위메프', 'title': '오뚜기참치/고추/야채 150GX10개 차이결제시', 'price': '10,040원/무료', 'tu': '2', 'td': '0'}
{'mall': '옥션', 'title': 'LG 통돌이 TR12BL', 'price': '361,000원 무료', 'tu': '1', 'td': '0'}
{'mall': '무신사', 'title': '스탠다드 깅엄 체크 린넨 셔츠', 'price': '9,900원/무료', 'tu': '1', 'td': '0'}
{'mall': '지마켓', 'title': '육아서 말로 때린 상처가 더 아프다', 'price': '11480원/무배', 'tu': '0', 'td': '7'}
{'mall': '쿠팡', 'title': '스윙칩 60g 12개', 'price': '9,900원 무료배송', 'tu': '7', 'td': '0'}
{

{'mall': '하이마트', 'title': '로보락 S5max', 'price': '559,000/무료', 'tu': '3', 'td': '0'}
{'mall': '네이버N샵', 'title': '바이타믹스 V1200I', 'price': '657,710/무료', 'tu': '13', 'td': '2'}
{'mall': '티몬', 'title': '장인의 소고기 육포 1봉', 'price': '990원/무배', 'tu': '1', 'td': '0'}
{'mall': '네이버N샵', 'title': '골든서클 골든패쉬 250ml x 12개', 'price': '200원/3,000원', 'tu': '24', 'td': '0'}
{'mall': 'G마켓', 'title': '테팔 매직핸즈 프렌치컬레션 멀티 5P세트', 'price': '62,900원/무료', 'tu': '0', 'td': '4'}
{'mall': '위메프', 'title': ') 삼성 ddr4 8g pc4 21300', 'price': '30,500/2300', 'tu': '3', 'td': '0'}
{'mall': '옥션', 'title': '제주 미니밤호박 가정용 2kg 8개내외', 'price': '9,500원/무료', 'tu': '3', 'td': '0'}
{'mall': '위메프', 'title': '라이젠9 3900x', 'price': '669,000원/무료', 'tu': '4', 'td': '0'}
{'mall': '카카오톡딜', 'title': '마비스치약세트 4종', 'price': '39,000원~/무배', 'tu': '8', 'td': '1'}
{'mall': '위메프', 'title': 'ASUS EX A320M-GAMING 메인보드', 'price': '83,000원/2,500원', 'tu': '1', 'td': '0'}
{'mall': '네이버쇼핑', 'title': '개사료,고양이사료 500g/1,5kg/2kg 샘플', 'price': '2,500원부터/무료', 't

In [140]:
url = 'http://www.ppomppu.co.kr/zboard/zboard.php?id=ppomppu'

urls = list()
seen = list()

urls.append(url)

while urls:
    seed = urls.pop(0)
    seen.append(seed)

    dom = BeautifulSoup(download(seed).text, 'lxml')
    for _ in [_['href']
              for _ in dom.select('''
              tr.list0 a[href], tr.list1 a[href], #page_list a[href]
              ''')
              if re.match(r'(?:https?:/)?/\w+(?:[./]\w+)+', _['href'])]:
        newUrls = urljoin(seed, _)
        if newUrls not in urls and newUrls not in seen:
            urls.append(newUrls)
    print(len(urls), len(seen))
    if len(seen) > 10: break

10 1
10 2
9 3
8 4
7 5
6 6
5 7
4 8
3 9
2 10
11 11


In [155]:
url = 'http://www.ppomppu.co.kr/zboard/zboard.php?'

params = {
    'id': 'freeboard'
}

resp = download(url, params, headers)
dom = BeautifulSoup(resp.text, 'lxml')

In [165]:
resp = download(url, params, headers)
dom = BeautifulSoup(resp.text, 'lxml')

urls = list()
seen = list()

urls.append({'url':url, 'depth':1})

# for _ in dom.select('.list_title')[1:]:
#     if _.find_parent()['href'].startswith('v'):
#         print(urljoin(url, _.find_parent()['href']))
#         resp = download(urljoin(url, _.find_parent()['href']))
#         dom = BeautifulSoup(resp.text, 'lxml')
#         print(dom.select_one('.board-contents').name)
for _ in dom.select('''
    .list0 > td:nth-of-type(3) > a,
    .list1 > td:nth-of-type(3) > a,
    #page_list a
    '''):
    if _['href'][0] in ['v', '/']:
        print(urljoin(url, _['href']))
        resp = download(urljoin(url, _['href']))
        dom = BeautifulSoup(resp.text, 'lxml')
        if dom.select_one('.board-contents') != None:
            print(dom.select_one('.board-contents').name)

# resp = download(url, params, headers)
# dom = BeautifulSoup(resp.text, 'lxml')
    
# for _ in dom.select('#page_list a'):
#     if _['href'].startswith('/'):
#         print(urljoin(url, _['href']))
#         resp = download(urljoin(url, _['href']))
#         dom = BeautifulSoup(resp.text, 'lxml')
#         if dom.select_one('.board-contents') != None:
#             print(dom.select_one('.board-contents').name)

http://www.ppomppu.co.kr/zboard/view.php?id=freeboard&page=1&divpage=1319&no=7080266
td
http://www.ppomppu.co.kr/zboard/view.php?id=freeboard&page=1&divpage=1319&no=7080264
td
http://www.ppomppu.co.kr/zboard/view.php?id=freeboard&page=1&divpage=1319&no=7080262
td
http://www.ppomppu.co.kr/zboard/view.php?id=freeboard&page=1&divpage=1319&no=7080260
td
http://www.ppomppu.co.kr/zboard/view.php?id=freeboard&page=1&divpage=1319&no=7080258
td
http://www.ppomppu.co.kr/zboard/view.php?id=freeboard&page=1&divpage=1319&no=7080256
td
http://www.ppomppu.co.kr/zboard/view.php?id=freeboard&page=1&divpage=1319&no=7080254
td
http://www.ppomppu.co.kr/zboard/view.php?id=freeboard&page=1&divpage=1319&no=7080252


KeyboardInterrupt: 

In [171]:
url = 'http://www.ppomppu.co.kr/zboard/zboard.php'

params = {
    'id':'freeboard',
    'page':1
}

resp = download(url, params, headers)
dom = BeautifulSoup(resp.text, 'lxml')

urls = list()
seen = list()

urls.append({'url':url, 'depth':1})

while urls:
    seed = urls.pop(0)
    
    if seed['depth'] > 2:
        break
        
    resp = download(seed['url'])
    dom = BeautifulSoup(resp.text, 'html.parser')
    
    seen.append(seed)
    
    for _ in dom.select('''
        .list0 > td:nth-of-type(3) > a,
        .list1 > td:nth-of-type(3) > a,
        #page_list a'''):
        if _['href'][0] in ['v', '/']:
            newurl = urljoin(url, _['href'])
            if newurl not in [_['url'] for _ in urls] and\
               newurl not in [_['url'] for _ in seen]:
                urls.append({'url':newurl,
                             'depth':seed['depth']+1})
            # 여기까지가 크롤링
            # 여기서부터가 스크래핑
            if dom.select_one('.board-contents') != None:
                print(dom.select_one('.board-contents').name)
    
# resp = download(url, params, headers)
# dom = BeautifulSoup(resp.text, 'lxml')
# for _ in dom.select('#page_list a'):
#     if _['href'].startswith('/'):
#         print(urljoin(url, _['href']))
#         resp = download(urljoin(url, _['href']))
#         dom = BeautifulSoup(resp.text, 'lxml')
#         if dom.select_one('.board-contents') != None:
#             print(dom.select_one('.board-contents').name)
#     print(_.text.strip())

In [ ]:
http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard

In [12]:
url = 'http://www.ppomppu.co.kr/zboard/zboard.php'

params = {
    'id':'freeboard'
}
headers = {
    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'
}

In [45]:
urls = list()
seen = list()

urls.append({'url':url, 'depth':1})

while urls:
    seed = urls.pop(0)
    
    if seed['depth'] > 2:
        break
        
    resp = download(seed['url'], params, headers)
    dom = BeautifulSoup(resp.text, 'html.parser')
    
    seen.append(seed)
    
    for _ in dom.select('.list0 > td:nth-of-type(3) a, .list1 > td:nth-of-type(3) a, #page_list a'):
#         print(_)
    
        if _['href'][0] in ['v', '/']:
            newurl = urljoin(url, _['href'])
            print(newurl)
            if newurl not in [_['url'] for _ in urls] and newurl not in [_['url'] for _ in seen]:
                urls.append({'url':newurl,
                             'depth':seed['depth']+1})
            # 여기까지가 크롤링
            # 여기서부터가 스크래핑
            
            
             #print(dom.select_one('.board-contents'))
            if dom.select_one('.board-contents') != None:
                print(dom.select_one('.board-contents').name)
    
# resp = download(url, params, headers)
# dom = BeautifulSoup(resp.text, 'lxml')
# for _ in dom.select('#page_list a'):
#     if _['href'].startswith('/'):
#         print(urljoin(url, _['href']))
#         resp = download(urljoin(url, _['href']))
#         dom = BeautifulSoup(resp.text, 'lxml')
#         if dom.select_one('.board-contents') != None:
#             print(dom.select_one('.board-contents').name)
#     print(_.text.strip())

http://www.ppomppu.co.kr/zboard/view.php?id=freeboard&page=1&divpage=1319&no=7080346
http://www.ppomppu.co.kr/zboard/view.php?id=freeboard&page=1&divpage=1319&no=7080344
http://www.ppomppu.co.kr/zboard/view.php?id=freeboard&page=1&divpage=1319&no=7080342
http://www.ppomppu.co.kr/zboard/view.php?id=freeboard&page=1&divpage=1319&no=7080340
http://www.ppomppu.co.kr/zboard/view.php?id=freeboard&page=1&divpage=1319&no=7080338
http://www.ppomppu.co.kr/zboard/view.php?id=freeboard&page=1&divpage=1319&no=7080336
http://www.ppomppu.co.kr/zboard/view.php?id=freeboard&page=1&divpage=1319&no=7080334
http://www.ppomppu.co.kr/zboard/view.php?id=freeboard&page=1&divpage=1319&no=7080332
http://www.ppomppu.co.kr/zboard/view.php?id=freeboard&page=1&divpage=1319&no=7080330
http://www.ppomppu.co.kr/zboard/view.php?id=freeboard&page=1&divpage=1319&no=7080328
http://www.ppomppu.co.kr/zboard/view.php?id=freeboard&page=1&divpage=1319&no=7080326
http://www.ppomppu.co.kr/zboard/view.php?id=freeboard&page=1&divp

KeyboardInterrupt: 

In [38]:
dom.select('table.pic_bg table table > tr > td:nth-of-type(1)')

[<td align="left" class="han" valign="top">
 <p> </p>
 <p>
 <img alt="66db2d48acd63235cff7260c7569e1433d8b4b14.jpg" border="0" src="//cdn.ppomppu.co.kr/zboard/data3/2020/0721/20200721164616_nwofciuh.jpg" style="width: 640px; height: 1708px;"/>
 </p>
 <p> </p>
 <p>힙합하는 가수중에는 크라운제이? 얘빼고는 </p>
 <p> </p>
 <p>거의 공익인듯 하네요. </p>
 <p> </p>
 <p>이런거보면 옥택연 같은 친구들이 정말 대단한듯..</p>
 <p> </p>
 <p> </p><!--"&lt;--></td>]